<a href="https://colab.research.google.com/github/sachins301/UTA-Distributed-Computing/blob/main/UTA_Spark_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Spark and dependencies

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.2-bin-hadoop3"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [4]:
# Unpack the json dump
import shutil
shutil.unpack_archive('/content/json_dumps.zip', '/content/json_dumps/')

In [5]:
# prompt: write a spark code to read jsons into a dataframe

df = spark.read.json("/content/json_dumps/*.json")
df.show()


+--------------------+
|                Siri|
+--------------------+
|{1.3, http://www....|
|{1.3, http://www....|
|{1.3, http://www....|
|{1.3, http://www....|
|{1.3, http://www....|
|{1.3, http://www....|
|{1.3, http://www....|
|{1.3, http://www....|
|{1.3, http://www....|
|{1.3, http://www....|
|{1.3, http://www....|
|{1.3, http://www....|
|{1.3, http://www....|
|{1.3, http://www....|
|{1.3, http://www....|
|{1.3, http://www....|
|{1.3, http://www....|
|{1.3, http://www....|
|{1.3, http://www....|
|{1.3, http://www....|
+--------------------+
only showing top 20 rows



In [6]:
df.count()

247